In [1]:
param_len = 5

list_url= ['https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-desc/sort-price-desc?page='+str(_+1)+'#prod-list' for _ in range(param_len)]

print('总数量：' + str(len(list_url)))
print()

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_part = html.xpath('//ul[@data-gtm-product-list-type="Search Results"]/li[@class="js-product-list-item"]')
                
                # = = = = = = = = = = = = = = =
                
                list_sku = [part.xpath('./@data-id')[0].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_title = [part.xpath('./@data-name')[0].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.carid.com' + part.xpath('./@data-url')[0].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                list_src = [part.xpath('./@data-src')[0].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'status': 'ok',
                                        'Menu': url,
                                        'Url': list_url,
                                        'Sku': list_sku,
                                        'Title': list_title,
                                        'Src': list_src})
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Menu': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./menu_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：5

https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-desc/sort-price-desc?page=5#prod-list  <->  [ok] - 剩余数量：0 - 当前时间：15:28:12
https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-desc/sort-price-desc?page=1#prod-list  <->  [ok] - 剩余数量：0 - 当前时间：15:28:12
https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-desc/sort-price-desc?page=2#prod-list  <->  [ok] - 剩余数量：0 - 当前时间：15:28:12
https://www.carid.com/search/NGK+Laser+Iridium+Spark+Plugs/code-ccb8ce4eee843e4eb433d7a9a8931e0d/queryId-420738fe115bb7ad7af949d2688d7e1a/&redirect=1%2Fsort-price-desc/sort-price-desc?page=3#prod-list  <->  [ok] - 剩余数量：0 - 当前时间：15:28:13
https://www.carid.com/search/NGK+Laser+Iridiu